In [4]:
from gurobipy import *
import random

def RINScallback(model, where):
    if where == GRB.Callback.MIPNODE:
        # MIP node callback
        if model.cbGet(GRB.Callback.MIPNODE_NODCNT) % 100 == 0 and \
                model.cbGet(GRB.Callback.MIPNODE_STATUS) == GRB.OPTIMAL:
            submodel = model.copy()
            suby = submodel.getVars()
            # 获得节点松弛解
            yrelaxation = model.cbGetNodeRel(model._y)
            # 固定变量取值
            for i in range(model._N):
                if abs(yrelaxation[i]) < 0.01:
                    suby[i].ub = 0
                elif abs(yrelaxation[i] - 1) < 0.01:
                    suby[i].lb = 1
            submodel.setParam(GRB.Param.TimeLimit, 30)
            submodel.optimize()
            if submodel.objval > model.cbGet(GRB.Callback.MIPNODE_OBJBST):
                # 将解传递给原模型
                for i in range(model._N):
                    if abs(suby[i].x) < 0.001:
                        model.cbSetSolution(model._y[i], 0.0)
                    elif abs(suby[i].x - 1) < 0.001:
                        model.cbSetSolution(model._y[i], 1.0)



In [2]:
try:
    random.seed(1)
    N = 10
    # 随机产生线段权重
    random.seed(1)
    Cmatrix = {(i, j): random.randint(0, 100) for i in range(N) for j in range(N)}
    m = Model('MaximumCut')
    # 添加变量
    y = m.addVars(N, vtype=GRB.BINARY, name='y')
    # 构造目标函数
    obj = QuadExpr()
    for i in range(N):
        for j in range(N):
            obj = obj + Cmatrix[i, j] * (y[i] + y[j] - 2 * y[i] * y[j])
    m.setObjective(0.5 * obj, -1)
    # 设置求解时间
    m.Params.TimeLimit = 600
    # 外部变量
    m._y = y
    m._N = N
    # 求解
    m.optimize(RINScallback)
    # 获得目标值和变量值
    print("Obj = ", m.ObjVal)
    for i in range(N):
        print(y[i].VarName, ' = ', y[i].x)

except GurobiError:
    print('Error reported')

Using license file /Users/xuning/gurobi.lic
Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 0 rows, 10 columns and 0 nonzeros
Model fingerprint: 0x360fd92f
Model has 54 quadratic objective terms
Variable types: 0 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [4e+02, 6e+02]
  QObjective range [6e+00, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective -0.0000000
Presolve time: 0.00s
Presolved: 45 rows, 55 columns, 135 nonzeros
Variable types: 0 continuous, 55 integer (55 binary)

Root relaxation: objective -2.541500e+03, 17 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    